In [ ]:
from langchain.tools import tool

In [ ]:
import random
from datetime import datetime, timedelta


def random_datetime_next_3_months():
    """Return a random datetime within the next 3 months (≈90 days)."""
    now = datetime.now()
    end = now + timedelta(days=90)  # approximate 3 months
    start_ts = now.timestamp()
    end_ts = end.timestamp()
    random_ts = random.uniform(start_ts, end_ts)
    return datetime.fromtimestamp(random_ts)


def generate_departure_arrival_pair(min_hours=1, max_hours=6):
    """Generate a departure and arrival pair.
    Arrival is between min_hours and max_hours after departure."""
    departure = random_datetime_next_3_months()
    delta_hours = random.uniform(min_hours, max_hours)
    arrival = departure + timedelta(hours=delta_hours)
    return departure, arrival

In [ ]:
[generate_departure_arrival_pair() for _ in range(3)]

[(datetime.datetime(2025, 11, 19, 0, 47, 17, 764309),
  datetime.datetime(2025, 11, 19, 5, 25, 52, 822790)),
 (datetime.datetime(2026, 1, 26, 8, 46, 49, 32676),
  datetime.datetime(2026, 1, 26, 14, 25, 9, 299027)),
 (datetime.datetime(2025, 12, 29, 22, 1, 13, 474749),
  datetime.datetime(2025, 12, 30, 2, 37, 44, 669631))]

In [ ]:
# from langgraph.graph import MessagesState
from langchain.agents import AgentState, create_agent
from langchain.agents.middleware import ModelRequest, dynamic_prompt
from langchain.messages import HumanMessage

## List next flight details


In [ ]:
@dynamic_prompt
def personalized_prompt(request: ModelRequest) -> str:
    user_id = request.state.get("user_id", None)
    return f"You are a helpful travel assistant assistant. User's id is {user_id}"

In [ ]:
@tool
def check_flight_status(flight_number: str) -> dict:
    """Check the status of a flight given its flight number.

    Args:
        flight_number (str): The flight number to check.
    Returns:
        dict: A dictionary containing flight status information. Dict keys include 'status', 'departure', 'arrival', and 'gate'.

    """
    departure, arrival = generate_departure_arrival_pair()
    # Dummy implementation for illustration
    return {
        "status": random.choice(["On Time", "Delayed", "Departed", "Boarding"]),
        "flight_number": flight_number,
        "departure": departure.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "arrival": arrival.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "gate": f"{random.choice(['A','B','C'])}{random.randint(1,30)}",
    }


@tool
def get_user_flight_revervations(user_id: str) -> list[str]:
    """Get a list of flight reservations for a given user.

    Args:
        user_id (str): The user ID to get reservations for.
    Returns:
        list[str]: A list of flight numbers
    """

    # Dummy implementation for illustration
    return [
        f"{random.choice(['AA', 'DL', 'UA', 'SW'])}{random.randint(100, 999)}"
        for _ in range(3)
    ]

In [ ]:
class CustomState(AgentState):
    user_id: str

In [ ]:
agent = create_agent(
    model="gpt-4o",
    tools=[check_flight_status, get_user_flight_revervations],
    state_schema=CustomState,
    middleware=[personalized_prompt],
)

In [ ]:
user_id = "user_1"
messages = [HumanMessage(content="What's the status of my next flight?")]

config = {"configurable": {"thread_id": "1", "user_id": user_id}}
response = agent.invoke({"messages": messages, "user_id": user_id}, config=config)

# for m in response["messages"]:
#     m.pretty_print()

In [ ]:
messages[0].pretty_print(), response["messages"][-1].pretty_print()

================================ Human Message =================================

What's the status of my next flight?
================================== Ai Message ==================================

Here are the statuses of your upcoming flights:

1. **Flight SW881**
   - **Status:** Departed
   - **Departure:** 2025-12-31 at 10:00 AM (UTC)
   - **Arrival:** 2025-12-31 at 03:49 PM (UTC)
   - **Gate:** B2

2. **Flight SW910**
   - **Status:** Boarding
   - **Departure:** 2026-01-29 at 07:34 PM (UTC)
   - **Arrival:** 2026-01-29 at 09:01 PM (UTC)
   - **Gate:** A25

3. **Flight UA985**
   - **Status:** Delayed
   - **Departure:** 2026-02-11 at 10:51 PM (UTC)
   - **Arrival:** 2026-02-12 at 02:36 AM (UTC)
   - **Gate:** A9

Your next flight is SW910, currently in the boarding stage. Safe travels!


(None, None)

## Change passenger name for next flight


In [ ]:
@tool
def change_reservation_name(flight_number: str, new_name: str, user_id: str) -> str:
    """Change the name on a flight reservation.

    Args:
        flight_number (str): The flight number to change the name on.
        new_name (str): The new name to set on the reservation.
        user_id (str): The user ID making the change.
    Returns:
        str: Confirmation message.
    """
    return f"Name on reservation for flight {flight_number} has been changed to {new_name} for user {user_id}."

In [ ]:
agent = create_agent(
    model="gpt-4o",
    tools=[check_flight_status, get_user_flight_revervations, change_reservation_name],
    state_schema=CustomState,
    middleware=[personalized_prompt],
)

In [ ]:
user_id = "user_1"
messages = [
    HumanMessage(content="Change the reservation name of my next flight to Jane Doe?")
]

config = {"configurable": {"thread_id": "1", "user_id": user_id}}
response = agent.invoke({"messages": messages, "user_id": user_id}, config=config)

# for m in response["messages"]:
#     m.pretty_print()

In [ ]:
messages[0].pretty_print(), response["messages"][-1].pretty_print()

================================ Human Message =================================

Change the reservation name of my next flight to Jane Doe?
================================== Ai Message ==================================

The name on your flight reservation for flight AA587 has been successfully changed to Jane Doe.


(None, None)

## Add baggage


In [ ]:
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command, interrupt

In [ ]:
@tool
def add_baggage(flight_number: str, baggage_count: int, user_id: str) -> str:
    """Add baggage to a flight reservation.

    Args:
        flight_number (str): The flight number to add baggage to.
        baggage_count (int): The number of bags to add.
        user_id (str): The user ID making the change.
    Returns:
        str: Confirmation message.
    """

    cost = baggage_count * 30  # Assume $30 per bag

    print("------ adding baggage")

    return f"Added {baggage_count} bags to reservation for flight {flight_number} for user {user_id}. Will cost ${cost}."

In [ ]:
checkpointer = InMemorySaver()
agent = create_agent(
    model="gpt-4o",
    tools=[
        check_flight_status,
        get_user_flight_revervations,
        change_reservation_name,
        add_baggage,
    ],
    state_schema=CustomState,
    middleware=[
        personalized_prompt,
        HumanInTheLoopMiddleware(
            interrupt_on={"add_baggage": {"allowed_decisions": ["approve", "reject"]}}
        ),
    ],
    checkpointer=checkpointer,
)

In [ ]:
user_id = "user_1"
messages = [HumanMessage(content="Add three bags to my next flight reservation.")]

config = {"configurable": {"thread_id": "1", "user_id": user_id}}
response = agent.invoke({"messages": messages, "user_id": user_id}, config=config)

# for m in response["messages"]:
#     m.pretty_print()

In [ ]:
# list(agent.get_state_history(config))
state_before_resume = next(agent.get_state_history(config))

In [ ]:
response["__interrupt__"]

[Interrupt(value={'action_requests': [{'name': 'add_baggage', 'args': {'flight_number': 'UA817', 'baggage_count': 3, 'user_id': 'user_1'}, 'description': "Tool execution requires approval\n\nTool: add_baggage\nArgs: {'flight_number': 'UA817', 'baggage_count': 3, 'user_id': 'user_1'}"}], 'review_configs': [{'action_name': 'add_baggage', 'allowed_decisions': ['approve', 'reject']}]}, id='6da5dd6d8f783923d9d2539d00094a4c')]

### User rejects


In [ ]:
response = agent.invoke(
    Command(resume={"decisions": [{"type": "reject"}]}), config=config
)
# for m in response["messages"]:
#     m.pretty_print()

In [ ]:
messages[0].pretty_print(), response["messages"][-1].pretty_print()

================================ Human Message =================================

Add three bags to my next flight reservation.
================================== Ai Message ==================================

It looks like you're unable to add baggage to your next flight reservation at this time. If there's anything else I can assist you with, please let me know!


(None, None)

### User accepts


In [ ]:
# Time travelling to before the resume. This time resume with approval.

new_config = agent.update_state(state_before_resume.config, values={})

response = agent.invoke(
    Command(resume={"decisions": [{"type": "approve"}]}), config=new_config
)
# for m in response["messages"]:
#     m.pretty_print()

------ adding baggage


In [ ]:
messages[0].pretty_print(), response["messages"][-1].pretty_print()

================================ Human Message =================================

Add three bags to my next flight reservation.
================================== Ai Message ==================================

I have added 3 bags to your next flight reservation (Flight UA817). This will cost you $90.


(None, None)

## User preferences as memories


In [ ]:
from langgraph.store.memory import InMemoryStore

store = InMemoryStore()

namespace_for_memory = ("users", user_id)
store.put(
    namespace_for_memory,
    "preferences",
    {"currency": "AUD"},
)

In [ ]:
@tool
def add_baggage_with_currency(
    flight_number: str, baggage_count: int, user_id: str, currency: str
) -> str:
    """Add baggage to a flight reservation.

    Args:
        flight_number (str): The flight number to add baggage to.
        baggage_count (int): The number of bags to add.
        user_id (str): The user ID making the change.
        currency (str): The currency to charge in.
    Returns:
        str: Confirmation message.
    """

    cost = baggage_count * 30  # Assume $30 per bag
    conversion_rates = {
        ("USD", "AUD"): 1.5,
        ("AUD", "USD"): 0.67,
    }
    cost = cost * conversion_rates.get(("USD", currency), 1)

    return f"Added {baggage_count} bags to reservation for flight {flight_number} for user {user_id}. Will cost ${cost} {currency}."

In [ ]:
@dynamic_prompt
def user_preferences_prompt(request: ModelRequest) -> str:
    user_id = request.state.get("user_id", None)
    namespace = ("users", user_id)
    store = request.runtime.store

    preferences = store.get(namespace, "preferences")
    return f"These are the user's preferences: {preferences}"

In [ ]:
checkpointer = InMemorySaver()
agent = create_agent(
    model="gpt-4o",
    tools=[
        check_flight_status,
        get_user_flight_revervations,
        change_reservation_name,
        add_baggage_with_currency,
    ],
    state_schema=CustomState,
    middleware=[
        personalized_prompt,
        user_preferences_prompt,
        HumanInTheLoopMiddleware(
            interrupt_on={"add_baggage": {"allowed_decisions": ["approve", "reject"]}}
        ),
    ],
    checkpointer=checkpointer,
    store=store,
)

In [ ]:
user_id = "user_1"
messages = [HumanMessage(content="Add three bags to my next flight reservation.")]

config = {"configurable": {"thread_id": "1", "user_id": user_id}}
response = agent.invoke({"messages": messages, "user_id": user_id}, config=config)

# Accept the addition of baggage
response = agent.invoke(
    Command(resume={"decisions": [{"type": "approve"}]}), config=config
)

# for m in response["messages"]:
#     m.pretty_print()

------ adding baggage


In [ ]:
messages[0].pretty_print(), response["messages"][-1].pretty_print()

================================ Human Message =================================

Add three bags to my next flight reservation.
================================== Ai Message ==================================

Three bags have been added to your next flight reservation (UA368) and will cost $135.00 AUD. Safe travels!


(None, None)

## Multi-agent RAG


### Tool Calling


In [ ]:
@tool
def get_knowledge_from_structured_data(query: str) -> str:
    """Get knowledge from structured data. Access flight pricing, availability, and more.
    The structured data schema is:
    ```
    {
        "price": "numeric",
        "currency": "string",
        "fare_class": "categorical",
        "seats_left": "numeric",
        "not_used_for_rag": ["raw_html", "debug_info"]
    }
    ```

    Args:
        query (str): The query to get knowledge for.
    Returns:
        str: The knowledge retrieved.
    """
    return f"Knowledge for query '{query}' for user {user_id}: [Structured Data Result]"


@tool
def get_knowledge_from_unstructured_data(query: str) -> str:
    """Get knowledge from unstructured data. Access information about visas, travel restrictions, and more.

    Args:
        query (str): The query to get knowledge for.
    Returns:
        str: The knowledge retrieved.
    """
    return f"Knowledge for query '{query}' for user {user_id}: China requires a visa for most travelers. For more information, visit https://www.totallymadeupwebsite.com/china-visa-info."

In [ ]:
rag_agent = create_agent(
    model="gpt-4o",
    tools=[
        get_knowledge_from_structured_data,
        get_knowledge_from_unstructured_data,
    ],
    system_prompt="Use the tools to answer user queries about travel using both structured and unstructured data. Provide supporting references and urls.",
)


@tool(
    "retrieval_augmented_generation_agent",
    description="Agent that uses retrieval augmented generation to answer user queries using both structured and unstructured data.",
)
def rag_agent_call(query: str) -> str:
    result = rag_agent.invoke({"messages": [HumanMessage(content=query)]})
    return result["messages"][-1].content

In [ ]:
checkpointer = InMemorySaver()
agent = create_agent(
    model="gpt-4o",
    tools=[
        check_flight_status,
        get_user_flight_revervations,
        change_reservation_name,
        add_baggage_with_currency,
        rag_agent_call,
    ],
    state_schema=CustomState,
    middleware=[
        personalized_prompt,
        user_preferences_prompt,
        HumanInTheLoopMiddleware(
            interrupt_on={"add_baggage": {"allowed_decisions": ["approve", "reject"]}}
        ),
    ],
    checkpointer=checkpointer,
    store=store,
)

In [ ]:
user_id = "user_1"
messages = [HumanMessage(content="What are China's visa restrictions.")]

config = {"configurable": {"thread_id": "1", "user_id": user_id}}
response = agent.invoke({"messages": messages, "user_id": user_id}, config=config)

# for m in response["messages"]:
#     m.pretty_print()

In [ ]:
messages[0].pretty_print(), response["messages"][-1].pretty_print()

================================ Human Message =================================

What are China's visa restrictions.
================================== Ai Message ==================================

China requires a visa for most travelers in 2023. For more detailed information, you can visit [China Visa Info](https://www.totallymadeupwebsite.com/china-visa-info).


(None, None)

In [ ]:
user_id = "user_1"
messages = [HumanMessage(content="How long can I stay in Brazil.")]

config = {"configurable": {"thread_id": "1", "user_id": user_id}}
response = agent.invoke({"messages": messages, "user_id": user_id}, config=config)

# for m in response["messages"]:
#     m.pretty_print()

In [ ]:
messages[0].pretty_print(), response["messages"][-1].pretty_print()

================================ Human Message =================================

How long can I stay in Brazil.
================================== Ai Message ==================================

I'm unable to retrieve specific information about the maximum stay in Brazil for tourists in 2023 right now. However, generally, tourists can stay in Brazil for up to 90 days within a 180-day period, although this can vary based on your nationality. For accurate and specific details, it's best to contact the Brazilian consulate or check official government resources.


(None, None)

### Hand-off
